# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [22]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
Import API key
from api_keys import gkey


In [23]:
!pip install citipy


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
cities_path="../cities.csv"
cities_df=pd.read_csv(cities_path)
cities_df.head

<bound method NDFrame.head of               Name    lat     lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
0            Korla  41.76   86.15     55.00        43          99        4.00   
1    Prince George  53.92 -122.75     30.20        74           1        5.82   
2        Tuatapere -46.13  167.68     53.01        66          95        7.00   
3        Mahébourg -20.41   57.70     66.20        93          75        3.36   
4          Ushuaia -54.80  -68.30     42.80        81          40        9.17   
..             ...    ...     ...       ...       ...         ...         ...   
554          Umluj  25.02   37.27     82.92        42           0        9.62   
555        Rāichūr  16.20   77.37     78.08        77          79        8.79   
556        Wanning  18.80  110.40     79.77        84          98       21.65   
557       La Unión  -5.40  -80.75     67.46        72          91        8.99   
558        Zambezi -13.54   23.10     67.37        43           0        8.30  

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
locations=cities_df[["lat","lng"]]
ratings=cities_df["Humidity"]
max_humidity=ratings.max()

In [29]:

gmaps.configure(api_key=gkey)
fig=gmaps.figure(center=(0,0), zoom_level=3)
heat_layer=gmaps.heatmap_layer(locations, weights=ratings,
                              dissipating=False, max_intensity=max_humidity,
                              point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
vacation_df=cities_df.copy()
vacation_df=vacation_df.loc[~(vacation_df["Max Temp"]<80) & (vacation_df["Cloudiness"]<30) & (vacation_df["Wind Speed"]<5)]
vacation_df


,Name,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hemisphere
131,Porto Nacional,-10.71,-48.42,80.60,78,20,3.36,BR,10/17/2020,South
212,Léogâne,18.51,-72.63,80.60,83,9,4.54,HT,10/17/2020,north
296,Bīrpur,26.52,87.02,82.53,67,0,1.30,IN,10/17/2020,north
316,Kokrajhar,26.40,90.27,82.67,69,8,3.29,IN,10/17/2020,north
332,Isla Mujeres,21.23,-86.73,82.40,78,20,3.36,MX,10/17/2020,north
341,Qandala,11.47,49.87,80.58,54,0,3.00,SO,10/17/2020,north
380,Brebes,-6.87,109.04,86.86,56,5,3.09,ID,10/17/2020,South
421,Tāndā,26.55,82.65,80.60,47,0,4.03,IN,10/17/2020,north
440,Ouro Sogui,15.61,-13.32,87.35,27,0,3.51,SN,10/17/2020,north
472,Pringsewu,-5.36,104.97,86.67,64,17,2.95,ID,10/17/2020,South


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [56]:
vacation_df["Hotel Name"]= ""
vacation_df["Hotel City"]=""
vacation_df.head()

,Name,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hemisphere,Hotel Name,Hotel City,Hotel Country
131,Porto Nacional,-10.71,-48.42,80.60,78,20,3.36,BR,10/17/2020,South,,,
212,Léogâne,18.51,-72.63,80.60,83,9,4.54,HT,10/17/2020,north,,,
296,Bīrpur,26.52,87.02,82.53,67,0,1.30,IN,10/17/2020,north,,,
316,Kokrajhar,26.40,90.27,82.67,69,8,3.29,IN,10/17/2020,north,,,
332,Isla Mujeres,21.23,-86.73,82.40,78,20,3.36,MX,10/17/2020,north,,,


In [60]:

for index, row in vacation_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    search_type = "lodging"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    
    params = {
        "location": f"{lat},{lng}",
        "radius":5000,
        "type": search_type,
        "key": gkey
    }
    
    #print(f"Retrieving Results for Index {index}.")
    response = requests.get(base_url, params=params)
    hotel_places = response.json()
    #print(json.dumps(hotel_places, indent=4,sort_keys=True))

   
    
    try:
        print("The closest hotel is:")
        print(hotel_places["results"][0]["name"])
        print("In the city of:")
        print(hotel_places["results"][0]["vicinity"])
        vacation_df.loc[index, 'Hotel Name']=hotel_places['results'][0]['name']
        vacation_df.loc[index, 'Hotel City']=hotel_places['results'][0]['vicinity']
      
        
    except:
        print("Missing field/result... skipping.")
        
    print("------------")


{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAIccwNb7mkwQ7V2bdw5BSEhs6Fbv7qqdUnVV0sRDaKTEwkmmgpZ5zcIXvzmvGnQdnGviG8WOqL7nfx-3srMTk9PXPRl1fxt-E-2vPa6SVR5MVVlPhMXYxZkZ6GWLpfwfbA8qWJT76nvVNIQ5d85EaPzR6qN84m-twgrjjjN8ZuLNYhTyFcQNIwSzIFz4cjXur4LhO0kWSdUt8AQ0y3t-6mJpPshdievV38GKZJdeDuSR0LdDD3BJEt4BJX4_-oi8qj3zmgxw6RDwsFPcLorEIDeWC31vWOky8KrahH0Q8VHSan_NwFPbaC02Myz1ggkHYsw6bu23GyXXkKQLb6xZY63CZRgos-JfDiBntLvSLI0vb1-ahHIMPf_G5MOKkgnBehIQafgr-MWgxwGAxaCJLwRsOxoU2WZFfdqG1dnayA3AHqpe7ZVJUDA",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -10.7097537,
                    "lng": -48.4076888
                },
                "viewport": {
                    "northeast": {
                        "lat": -10.7084048197085,
                        "lng": -48.4063396697085
                    },
                    "southwest": {
                        "lat": -10.71110278029

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAEI5PuiPbc26U3kxgv9a4t7-qAu1p6e3AsCDRQtY0eNw7cPeqlv5W7vB8y8OMyLqi26r75jXZ0IPSmEuHNdSLGzNlN0nT0FTqLZuhNkLEVuQSr-yGxfyWoF6dZ9fMoINiq63d3mR4nlj8fv7znbsPeIajrom6NwwViUczbaBSZ_iB7IRcSw4_rKbDAmfgMHxO8XLdlCBRS8lyFusGx7Re1pxLA4fNcZOGdVOqwOSr1QbAxSG5NwbkUz_R5MRwOrAkjyF38WV9l3XAPWMOuuUpVPW4gpZcuJahUkL5TeHhMSxuBeyXbhQSmcXnwu7M0uS6PXliJdRjFUAMedP9GKdqbSshuW7_dHmOZPabyr8WmsStfibPpFExEVtHK90gn8dDRIQuVBuTtWw8ZvMSF-1cFU7MhoUVERs6Xn73Pua5IHPJ75p4eTKKtM",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 18.4952338,
                    "lng": -72.6296255
                },
                "viewport": {
                    "northeast": {
                        "lat": 18.4966140802915,
                        "lng": -72.6283111697085
                    },
                    "southwest": {
                        "lat": 18.4939161197085,

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAACPD0WM7XCIRPDDoSyfrt18MTqrls2iXj6JvVl6w6yM41PfPHPRJ-5PxR-_CVrpJ16s8N0TFa4TTqLvT6ZgFrsxL_kc5f7BxPgOXcgkBS3k1FhjolGCaCGLajGytqPy55dGFrdSPRp0NbzX0Lji_0S8nUM1P4I-NxmXgnFwzcvfK4lhLpb_ejiV3CDAWIu29qwuZ9BgwEiU1a_lWcXwvXIY5l1K7vc1ca0OxC4W5Q-AbMLz5mE7CJpa_Gr2lYp1_Ssv3gzgPjoYOXeRi0xndYEVFJAffVXgfuomzXWoGbhMsuljj8HNedTHbQYaeamibxZ9RAYxIpgAaiPI8QuQ284b9JJfYmwGwDFJleOE3KjhiwQUB8reo5QMW8N4xWqOjPxIQ_3xYMRaOAX6FgzPN4hXiSRoUadB4cy38Pvbc24CtwgaXQs1YQdc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 26.5160012,
                    "lng": 87.01620640000002
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.5173127802915,
                        "lng": 87.0175637302915
                    },
                    "southwest": {
                        "lat": 26.514614819

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAKy5cr96G36xlJYfkyJJcoI66rQpaoXGxsFFpSvo6vAgcwGX9NB7hyb2Av-bDNg3RuK0tLN2nSjkjofJ6okxivX89015huuFE5EK6CxUvLbgt9CMtmZyORBRtA7GBy4_q3rc4jRG1p7iDKR7n_6QTLpyNAz7J3n_LykvkFPYp1Tcke60GudPekCan0YWotQoT1OPcx3FG-iKtcnlsH91G_C5N63XaaZXGZ3FP5Rs5SByDcx1Cg0OO15Gnw0txNfOR8-5QuW-m69maOUo9UZ9p-nGK-mNgpX0305giUYcNga9XpJq37eDXDjBkOvXSsLFC6AgEDmsVvYWlPlR_GoYhQ6dQUJIJzBXoIErGvikXNlI98XSbkJyb19fXGHBbYQqnBIQnZkH1KxRjOW_7pLwq4dSsxoUOFEWHwJhmfgUy2pIa9sllb8ewXM",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 26.4005183,
                    "lng": 90.26874459999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.4019200802915,
                        "lng": 90.2699986802915
                    },
                    "southwest": {
                        "lat": 26.399222119

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAF30q1EeQR3Cwm_UGmpQMiAUluPK6Ht2M_Gj2eAFtOFmnS-HNH61do6EJ8lbBWxDYDW4bOUvXtf8ZOddZA2peIyhJ5lNrGTjLWAoo-oEYLwvbk9tKyF16b47o09ALvyWfy7L91HnkYvXZbuaB8BQkofYn1zg755LFdTmkBVDdyar6AOVfE1qCMujlFY8jE4eeGuzzK82gB6KqCLKtT5hoiS03Z0aUnhWUgyhHc56vZJG6C9BCwfcK9-tOFHlOU53Uu9DSWdEJIbvR0VNFOr69GxnGahb4G5vkQ2dNlp4yszPTSiBuQUXda9i-qwXxNIyEIkpwIYWwGWX4SIuLPNnCfPi4zvX1tUuerYhJS7ZwyEkYZEkOo58_D7bcu_Frzwp-RIQWLEURTavR97B0zrxEeoonRoUkweGboVNrTa_vmN1vn3l8V6GmE8",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 21.209057,
                    "lng": -86.71991109999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 21.2104105302915,
                        "lng": -86.7185611197085
                    },
                    "southwest": {
                        "lat": 21.20771256

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAF16YUd3TV21yDxp5mDGakBEPZJh-ygt7ICp5rMm50THNfPSzVGR8t59yZomE1whH2VAvTRDxss5ah5YeVHtksf2M5lIdXW6pZ--t2CeaQPi0FYn8rs6OhHxNaDLJxj6pzbzQtKbV2_jyzbxnapMHyqo_EhXl-z0MIPExCZwRV_05tK_xdKmrZ-yOpkK4phZhIjv_zitioOwmdbQVfIYXMArwFdlo9tCc5D4XK9-wS0maO4X7td3iF3GUjd--9X1akAfoHbOq_eRCejurYPpxBeWGiiBdCjn5NGWUFk8Pco8qVEjP7ODzVLOtKuVLiNoADZXsJF3ofkJ4lSqTHFNBQxcH4Arv1GZrwmhcUzRSPZWeXfNMKJ0OWBx2lfRaAWRGhIQvsOD70nNZJgSic1tdXOHchoUYklTH4C5dGUdfFTsTmQ0qpvS9eg",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -6.8737257,
                    "lng": 109.0435066
                },
                "viewport": {
                    "northeast": {
                        "lat": -6.872472569708497,
                        "lng": 109.0450101302915
                    },
                    "southwest": {
                        "lat": -6.875170530291

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAKl-D81j3haYJKhrhNIJPrAJSMb5Qqj06l5vfJ9I9rVdajuBRWvbZAYsbEAJarsBWDS7BvED93tSz2-o0DG2qVxVeotfHiuLTB7vo-VS5ulZQrD54TWgENVNjWO7X7EUwgvy-wUw-NhcZNnFGEi0Q3IBph8iSi0bdSuy3lsikMt4vD1yOl3_0JnobqLgItN1bVpzrqsKnEqSxYknYLPtqHc9sR9A0YLuTvhKWyy5Z2xAG8pErG3uFmHmAoHmys061myYU8riYpR51Hp_NCRCzZn3bnmbvP_yb1_37f8v2NZNgUHdiwVYodGKxyTw_5S5ULtY6CLGDWqsA0sHUoLN6hucyPjwu-yeUtoWX3b4kqtsq_i1aH85GPiVdiqBvaY7FBIQWX-bndPvXGjrJ5V53895iBoUteteRE2ZlT2cK-CkM60iMuwyLVs",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 26.5556918,
                    "lng": 82.6573396
                },
                "viewport": {
                    "northeast": {
                        "lat": 26.5570133302915,
                        "lng": 82.6586809302915
                    },
                    "southwest": {
                        "lat": 26.5543153697085,
 

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 15.6096665,
                    "lng": -13.3202107
                },
                "viewport": {
                    "northeast": {
                        "lat": 15.6109935302915,
                        "lng": -13.3189099697085
                    },
                    "southwest": {
                        "lat": 15.6082955697085,
                        "lng": -13.3216079302915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Auberge Sogui",
            "place_id": "ChIJBbpXb2aR8g4RDb6U9LdYUHk",
            "plus_code": {
                "compound_code": "JM5H+VW Ouro Sogui, Senegal",
                "global_code": "7C78JM5H+VW"
            },
            "reference": "Ch

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAADqQsQRoVG0tOB6ScNZB61hjhs8slwlUaaW0PakjG7HhP5GpidHK19-qnNeE7jB-h52bX2mUTcVEkecZ-xQHH3WU18LTxR2qhX7gLUUJhLbN88vCak_hwXxMDU3_PSbhRkFsi3ozC9mKQbvsJefnx-2U3KNJ5qEib10FB-q3kfvbbdRy_Sz1HrRpAbZjlixwXKTTvYk5EM4UdN3giRcNAEXqCP4B2clu43gMak8AMUBp7WdJCc1rnRw51Dxte96a5llghJB-sfNe3KkLNULRBTx9IPh9J-taH7q3isXTqh4T6urvqhEeT4L07KYNwLYxK2DmGwlE9H9kJlr2YPUBeqg03vaTfeFJvttJbvaiLbwPUPIK513LDAJ6PuVWmkCjQRIQ-oO972Y2ni38nMBrO26l4hoUt_CXrIBUqSccbj2kwMghyHGBYyE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -5.356786,
                    "lng": 104.9836289
                },
                "viewport": {
                    "northeast": {
                        "lat": -5.355488869708497,
                        "lng": 104.9849676302915
                    },
                    "southwest": {
                        "lat": -5.3581868302915

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 16.5101114,
                    "lng": -14.6500371
                },
                "viewport": {
                    "northeast": {
                        "lat": 16.5119577302915,
                        "lng": -14.6486032197085
                    },
                    "southwest": {
                        "lat": 16.5092597697085,
                        "lng": -14.6513011802915
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "l'h\u00f4tel samassa",
            "place_id": "ChIJJbpdB2cTkw4RKcCurSTUxho",
            "plus_code": {
                "compound_code": "G86X+2X Ndioum, Senegal",
                "global_code": "7C87G86X+2X"
            },
            "reference": 

In [63]:
vacation_df["Hotel Country"]=vacation_df["Country"]
vacation_df.head()

,Name,lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,hemisphere,Hotel Name,Hotel City,Hotel Country
131,Porto Nacional,-10.71,-48.42,80.60,78,20,3.36,BR,10/17/2020,South,Vila Real Hotel,"Viela Vinte e Um, 10 - Setor Aeroporto, Porto ...",BR
212,Léogâne,18.51,-72.63,80.60,83,9,4.54,HT,10/17/2020,north,Family Health Ministries Guest House,Route Nationale 2,HT
296,Bīrpur,26.52,87.02,82.53,67,0,1.30,IN,10/17/2020,north,Maa Sherawali Sweet Content,Birpur,IN
316,Kokrajhar,26.40,90.27,82.67,69,8,3.29,IN,10/17/2020,north,BARBI HOTEL,"Tengapara, Kokrajhar",IN
332,Isla Mujeres,21.23,-86.73,82.40,78,20,3.36,MX,10/17/2020,north,Hotel La Joya Isla Mujeres,Secc. F Mz 43 L20 Carretera Garrafon Fracc Mar...,MX


In [64]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Hotel Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]
locations =vacation_df[["lat", "lng"]]
#ratings=vacation_df[["Humidity"]].astype(float)

In [65]:
# Add marker layer ontop of heat map
figs=gmaps.figure(zoom_level=.5, center=(20,30))
markers=gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
#heat_layer=gmaps.heatmap_layer(locations, weights=ratings, dissipating=False, max_intensity=100, point_radius=1.5)
fig.add_layer(heat_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))